In [1]:
import requests
import pandas as pd

# Danh sách các mã cổ phiếu

tickers = [
    "VRE.VN", "MWG.VN", "FPT.VN", "BID.VN", "HCM.VN", 
    "VNM.VN", "VCB.VN", "ACB.VN", "TCH.VN", "HPG.VN", 
    "SAB.VN", "BTP.VN", "TDM.VN", "NT2.VN", "HVN.VN", 
    "MBB.VN", "HII.VN", "SBT.VN", "VNG.VN", "QCG.VN", 
    "ASM.VN", "VTO.VN", "MSH.VN", "HAX.VN", 
    "VHC.VN", "GEX.VN", "SJD.VN", "TTF.VN", "VLP.VN", 
    "DTL.VN", "IJC.VN", "BVH.VN", "TDC.VN", "TTH.VN", 
    "EVG.VN", "CIG.VN", "DBD.VN", "NVT.VN", "BMI.VN", 
    "CVT.VN", "SCS.VN", "VCI.VN", "GDT.VN", "YEG.VN", 
    "D2D.VN", "MHC.VN", "SHB.VN", "LHG.VN", "TNI.VN",  
    "VCG.VN", "LGL.VN", "SAM.VN", "SMC.VN", "DPM.VN", 
    "AST.VN", "BHN.VN"
]



crumb = "adz0tLtBABS"

cookies = {
    "A1": "d=AQABBIa952gCEKgzbxthYhQX5HTuQ5RfucIFEgEBCAEK6WgPadTQ0iMA_eMDAAcIhr3naJRfucI&S=AQAAAloQl_xlt7N0NPllMhFhrrY",
    "A3": "d=AQABBIa952gCEKgzbxthYhQX5HTuQ5RfucIFEgEBCAEK6WgPadTQ0iMA_eMDAAcIhr3naJRfucI&S=AQAAAloQl_xlt7N0NPllMhFhrrY",
    "A1S": "d=AQABBIa952gCEKgzbxthYhQX5HTuQ5RfucIFEgEBCAEK6WgPadTQ0iMA_eMDAAcIhr3naJRfucI&S=AQAAAloQl_xlt7N0NPllMhFhrrY",
    "GUC": "AQEACAJo4LJpCEIcEQRQ&s=AQAAAC28q-NF&g=aN9lWA",
    "PRF": "t%3DVNM.VN%252B%255EVNINDEX.VN",
    "fes-ds-session": "pv%3D1"
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
}

modules = {
    "incomeStatementHistoryQuarterly": "incomeStatementHistory"
}

def extract_raw_values(entry):
    flat = {}
    for k, v in entry.items():
        if isinstance(v, dict):
            flat[k] = v.get("raw", v.get("fmt", None))
        else:
            flat[k] = v
    return flat

all_frames = []

# Lặp qua từng mã cổ phiếu trong danh sách
for ticker in tickers:
    for mod, inner_key in modules.items():
        url = f"https://query1.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?modules={mod}&crumb={crumb}"
        res = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        print(f"[{mod}] {ticker} status:", res.status_code)

        if not res.ok:
            print(res.text[:200])
            continue

        data = res.json()["quoteSummary"]["result"][0].get(mod, {})
        entries = data.get(inner_key, [])
        if not entries:
            print("⚠️ Không có dữ liệu cho:", mod)
            continue

        clean_entries = [extract_raw_values(e) for e in entries]
        df = pd.DataFrame(clean_entries)

        if "endDate" in df.columns:
            df["date"] = pd.to_datetime(df["endDate"], unit="s", errors="coerce").dt.strftime("%Y-%m-%d")
            df.set_index("date", inplace=True)

        df["ticker"] = ticker  # Thêm cột ticker
        all_frames.append(df)

# Gộp tất cả dữ liệu lại
final_df = pd.concat(all_frames, axis=0)

# Xoá cột trùng
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

# Xuất ra CSV
final_df.to_csv("financials_VN.csv", encoding="utf-8-sig")

print("✅ Đã lưu: financials_VN.csv")
print("📊 Số cột:", len(final_df.columns))
print("📅 Các kỳ:", list(final_df.index))



[incomeStatementHistoryQuarterly] VRE.VN status: 200
[incomeStatementHistoryQuarterly] MWG.VN status: 200
[incomeStatementHistoryQuarterly] FPT.VN status: 200
[incomeStatementHistoryQuarterly] BID.VN status: 200
[incomeStatementHistoryQuarterly] HCM.VN status: 200
⚠️ Không có dữ liệu cho: incomeStatementHistoryQuarterly
[incomeStatementHistoryQuarterly] VNM.VN status: 200
[incomeStatementHistoryQuarterly] VCB.VN status: 200
[incomeStatementHistoryQuarterly] ACB.VN status: 200
[incomeStatementHistoryQuarterly] TCH.VN status: 200
⚠️ Không có dữ liệu cho: incomeStatementHistoryQuarterly
[incomeStatementHistoryQuarterly] HPG.VN status: 200
[incomeStatementHistoryQuarterly] SAB.VN status: 200
[incomeStatementHistoryQuarterly] BTP.VN status: 200
[incomeStatementHistoryQuarterly] TDM.VN status: 200
[incomeStatementHistoryQuarterly] NT2.VN status: 200
[incomeStatementHistoryQuarterly] HVN.VN status: 200
[incomeStatementHistoryQuarterly] MBB.VN status: 200
[incomeStatementHistoryQuarterly] HII.